Exp2: X -> P1 -> Ro

where X = [Bulgarian, Greek, Serbian]

pivot = en
* bg - en
* (gr) el - en 
* sr - en
* en - ro

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/MyDrive/Colab Notebooks/MT_Project'

In [ ]:
# % pip install datasets
# % pip install transformers
# % pip install sentencepiece
# % pip install sacrebleu==1.5.1

In [4]:
import torch

import datasets
from datasets import load_dataset

import transformers
from transformers import MarianTokenizer, MarianMTModel, AutoTokenizer, AutoModelForSeq2SeqLM


In [5]:
TEST_INDEX_START = 12500 # 15000
TEST_INDEX_END = 15000 # 15000

In [ ]:

subsets = [
    { "dataset": "bg-ro", "source": "bg", "target": "ro", "model": "bg-en"},
    { "dataset": "el-ro", "source": "el", "target": "ro", "model": "grk-en"},
    { "dataset": "ro-sr", "source": "sr", "target": "ro", "model": ""}, # no model
]

datasets = {}
test_dataset = {}
for subset in subsets:
    datasets[subset["dataset"]] = load_dataset("setimes", subset["dataset"])
    test_dataset[subset["dataset"]] = {}
    test_dataset[subset["dataset"]][subset["source"]] = [translation[subset["source"]] for translation in datasets[subset["dataset"]]["train"]["translation"][TEST_INDEX_START:TEST_INDEX_END]]
    test_dataset[subset["dataset"]][subset["target"]] = [translation[subset["target"]] for translation in datasets[subset["dataset"]]["train"]["translation"][TEST_INDEX_START:TEST_INDEX_END]]

In [7]:
datasets["bg-ro"]["train"]["translation"][TEST_INDEX_START:TEST_INDEX_START+5]

[{'bg': 'SETimes: Смятате ли, че този вид проблеми не са вече толкова тежки?',
  'ro': 'SETimes: Consideraţi că lucrurile se îmbunătăţesc?'},
 {'bg': 'Как се промениха през последните няколко години?',
  'ro': 'Cum s-a modificat situaţia în ultimii ani?'},
 {'bg': 'Koптас: Относително погледнато, положението се подобрява, тъй като преди 20 години бе невъзможно да се говори за тези проблеми и турските интелектуалци не знаеха за съществуването на тези проблеми.',
  'ro': 'Koptas: Relativ vorbind, se îmbunătăţesc lucrurile, pentru că acum 20 de ani nu se putea vorbi despre aceste aspecte, iar intelectualii turci nu erau la curent cu aceste fenomene.'},
 {'bg': 'Само арменците защитаваха правата си и кюрдите защитаваха собствените си права, но сега е различно: сега съществува група интелектуалци [от всички етноси], които се противопоставят на този вид дискриминация.',
  'ro': 'Doar armenii îşi apărau propria cauză, iar kurzii pe a lor. Acum situaţiile se întrepătrund: acum există un grup c

In [8]:
print(len(test_dataset["bg-ro"]["bg"]))
test_dataset["bg-ro"]["bg"][:5]

2500


['SETimes: Смятате ли, че този вид проблеми не са вече толкова тежки?',
 'Как се промениха през последните няколко години?',
 'Koптас: Относително погледнато, положението се подобрява, тъй като преди 20 години бе невъзможно да се говори за тези проблеми и турските интелектуалци не знаеха за съществуването на тези проблеми.',
 'Само арменците защитаваха правата си и кюрдите защитаваха собствените си права, но сега е различно: сега съществува група интелектуалци [от всички етноси], които се противопоставят на този вид дискриминация.',
 'С тези идеи расте младото поколение, така че хората могат да разберат, че национализмът не е толкова добър, колкото твърди държавата.']

In [9]:
def use_GPU():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = use_GPU()

device

device(type='cuda')

In [10]:
BATCH_SIZE = 16

model_en_ro = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ro")
model_en_ro = model_en_ro.to(device)
tokenizerl_en_ro = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ro")

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

In [11]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

metric = load_metric("sacrebleu")

results = {}

# BG - EN - RO

In [12]:
test_dataset_bg = test_dataset["bg-ro"]["bg"]

model_bg_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bg-en")
model_bg_en = model_bg_en.to(device)
tokenizerl_bg_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bg-en")


batch_index = 0
predicted_bg_en = []
while batch_index < len(test_dataset_bg):
    if batch_index % 500 == 0:
        print("BG-EN batch_index:", batch_index)
    batch_bg = tokenizerl_bg_en(test_dataset_bg[batch_index:batch_index+BATCH_SIZE], return_tensors="pt", padding=True).to(device)
    gen_bg = model_bg_en.generate(**batch_bg)
    predicted_bg_en.extend(tokenizerl_bg_en.batch_decode(gen_bg, skip_special_tokens=True))

    batch_index += BATCH_SIZE


batch_index = 0
predicted_en_ro = []
while batch_index < len(predicted_bg_en):
    if batch_index % 500 == 0:
        print("EN-RO batch_index:", batch_index)
    batch_en = tokenizerl_en_ro(predicted_bg_en[batch_index:batch_index+BATCH_SIZE], return_tensors="pt", padding=True).to(device)
    gen_en = model_en_ro.generate(**batch_en)
    predicted_en_ro.extend(tokenizerl_en_ro.batch_decode(gen_en, skip_special_tokens=True))

    batch_index += BATCH_SIZE


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/973k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

BG-EN batch_index: 0
BG-EN batch_index: 2000
EN-RO batch_index: 0
EN-RO batch_index: 2000


In [17]:
predicted_bg_en_ro = predicted_en_ro
predicted_bg_en_ro[:5]

['SETimes: Credeţi că acest tip de problemă nu mai este atât de gravă?',
 'Cum s-au schimbat în ultimii ani?',
 'Koptas: Relativ vorbind, situaţia se îmbunătăţeşte deoarece acum 20 de ani era imposibil să vorbim despre aceste probleme, iar intelectualii turci nu ştiau despre existenţa acestor probleme.',
 'Numai armenii și-au apărat drepturile și kurzii și-au apărat propriile drepturi, dar acum este diferit: există acum un grup de intelectuali [din toate etniile] care se opun acestui tip de discriminare.',
 'Cu aceste idei creşte tânăra generaţie, astfel încât oamenii să poată înţelege că naţionalismul nu este la fel de bun precum declară statul.']

In [18]:
test_dataset_ro = test_dataset["bg-ro"]["ro"]
test_dataset_ro[:5]

['SETimes: Consideraţi că lucrurile se îmbunătăţesc?',
 'Cum s-a modificat situaţia în ultimii ani?',
 'Koptas: Relativ vorbind, se îmbunătăţesc lucrurile, pentru că acum 20 de ani nu se putea vorbi despre aceste aspecte, iar intelectualii turci nu erau la curent cu aceste fenomene.',
 'Doar armenii îşi apărau propria cauză, iar kurzii pe a lor. Acum situaţiile se întrepătrund: acum există un grup comun de intelectuali [de toate etniile] care se opune tuturor formelor de discriminare.',
 'O generaţie nouă creşte alături de aceste idei, astfel că oamenii pot vedea că naţionalismul nu este benefic, aşa cum zice statul.']

In [19]:
predictions, references = postprocess_text(predicted_bg_en_ro, test_dataset_ro)
bleu_score = metric.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_score['score']:.4f}")

results["bg_en_ro"] = bleu_score

BLEU: 36.4931


# EL (GRK) - EN - RO

In [20]:
test_dataset_el = test_dataset["el-ro"]["el"]

model_grk_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-grk-en")
model_grk_en = model_grk_en.to(device)
tokenizerl_grk_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-grk-en")


batch_index = 0
predicted_grk_en = []
while batch_index < len(test_dataset_el):
    if batch_index % 500 == 0:
        print("GRK-EN batch_index:", batch_index)
    batch_grk = tokenizerl_grk_en(test_dataset_el[batch_index:batch_index+BATCH_SIZE], return_tensors="pt", padding=True).to(device)
    gen_grk = model_grk_en.generate(**batch_grk)
    predicted_grk_en.extend(tokenizerl_grk_en.batch_decode(gen_grk, skip_special_tokens=True))

    batch_index += BATCH_SIZE


batch_index = 0
predicted_en_ro = []
while batch_index < len(predicted_grk_en):
    if batch_index % 500 == 0:
        print("EN-RO batch_index:", batch_index)
    batch_en = tokenizerl_en_ro(predicted_grk_en[batch_index:batch_index+BATCH_SIZE], return_tensors="pt", padding=True).to(device)
    gen_en = model_en_ro.generate(**batch_en)
    predicted_en_ro.extend(tokenizerl_en_ro.batch_decode(gen_en, skip_special_tokens=True))

    batch_index += BATCH_SIZE



Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/214M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

GRK-EN batch_index: 0
GRK-EN batch_index: 496
GRK-EN batch_index: 992
GRK-EN batch_index: 1488
GRK-EN batch_index: 1984
GRK-EN batch_index: 2480
EN-RO batch_index: 0
EN-RO batch_index: 496
EN-RO batch_index: 992
EN-RO batch_index: 1488
EN-RO batch_index: 1984
EN-RO batch_index: 2480


In [21]:
predicted_grk_en_ro = predicted_en_ro
predicted_grk_en_ro[:5]

['Șomajul a atins 11,9% în aprilie, potrivit previziunilor FMI, și este de aproximativ 15% în 2012.',
 'Există, de asemenea, preocupări cu privire la capacitatea guvernului de a menține noua politică.',
 '"Adevărata problemă... este credibilitatea sistemului politic," a declarat Green.',
 'Papandreou a făcut 180 de grade din programul socialist al partidului său, PASOK, a indignat câţiva membri.',
 '"Toate aceste [măsuri de austeritate] intră în conflict cu ceea ce a spus în timpul campaniei electorale... dar s-a dovedit flexibil," a afirmat Stournaras.']

In [22]:
test_dataset_ro = test_dataset["el-ro"]["ro"]
test_dataset_ro[:5]

['Rata şomajului a ajuns în luna aprilie la 11,9%, potrivit ANA, iar previziunile FMI afirmă că va atinge un maxim de aproape 15%, în 2012.',
 'De asemenea, există îngrijorări că guvernul nu va avea abilitatea de a-şi menţine noua politică.',
 '„Adevărata problemă... este credibilitatea sistemului politic”, a afirmat Glynos.',
 'Papandreou s-a întors la 180 de grade faţă de platforma socialistă a partidului său PASOK, ceea ce i-a deranjat pe unii membri.',
 '„Toate aceste lucruri [măsurile de austeritate] sunt în conflict cu ceea ce a spus el în campania pre-electorală... însă s-a dovedit a fi flexibil”, a spus Stournaras.']

In [23]:

predictions, references = postprocess_text(predicted_grk_en_ro, test_dataset_ro)
bleu_score = metric.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_score['score']:.4f}")

results["bg_en_ro"] = bleu_score

BLEU: 35.5486


# SR - EN - RO

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

test_dataset_sr = test_dataset["ro-sr"]["sr"]

model_sr_en = MarianMTModel.from_pretrained("opus-mt-uk-en-finetuned-sr-to-en/checkpoint-12500")
model_sr_en = model_sr_en.to(device)
tokenizerl_sr_en = MarianTokenizer.from_pretrained("opus-mt-uk-en-finetuned-sr-to-en/checkpoint-12500")


batch_index = 0
predicted_bg_en = []
while batch_index < len(test_dataset_sr):
    if batch_index % 500 == 0:
        print("SR-EN batch_index:", batch_index)
    batch_bg = tokenizerl_sr_en(test_dataset_sr[batch_index:batch_index+BATCH_SIZE], return_tensors="pt", padding=True).to(device)
    gen_bg = model_sr_en.generate(**batch_bg)
    predicted_bg_en.extend(tokenizerl_sr_en.batch_decode(gen_bg, skip_special_tokens=True))

    batch_index += BATCH_SIZE


batch_index = 0
predicted_en_ro = []
while batch_index < len(predicted_bg_en):
    if batch_index % 500 == 0:
        print("EN-RO batch_index:", batch_index)
    batch_en = tokenizerl_en_ro(predicted_bg_en[batch_index:batch_index+BATCH_SIZE], return_tensors="pt", padding=True).to(device)
    gen_en = model_en_ro.generate(**batch_en)
    predicted_en_ro.extend(tokenizerl_en_ro.batch_decode(gen_en, skip_special_tokens=True))

    batch_index += BATCH_SIZE



BG-EN batch_index: 0
BG-EN batch_index: 2000
EN-RO batch_index: 0
EN-RO batch_index: 2000


In [ ]:
predicted_en_ro[:5]

['Lemierres a declarat că BERD va încerca să stabilească un climat de investiţii bun, să încheie acorduri de liber schimb şi să asigure o abordare mai bună a împrumuturilor acordate în Europa de Sud-est întreprinderilor mici şi mijlocii.',
 'Cadrul celor dispăruţi şi încă impresionabili, spun liderii',
 'Belgradul a găzduit o conferinţă cu ocazia aniversării pauzei, care dintre fondatori a fost după Tito.',
 'De Biljana Pekusic, din Belgrad pentru Southeast European Times -- 7.09/11',
 'Miniştrii afacerilor externe şi şefii delegaţiei ţării a Reprezentanţilor de Interne din Popret cer construirea Adunării Sârbe de la Belgrad în 5 septembrie. [Reuters]']

In [ ]:
test_dataset["ro-sr"]["ro"][:5]

['Lemierres a afirmat că BERD va depune eforturi pentru crearea unui climat bun pentru investiţii, pentru încheierea Acordurilor de Comerţ Liber şi pentru a se asigura că întreprinderile mici şi mijlocii au acces mai uşor la împrumuturi în Europa de Sud-est.',
 'Mişcarea de nealiniere are încă influenţă, afirmă liderii',
 'Belgradul găzduieşte conferinţa aniversară a unei iniţiative co-lansate de fostul preşedinte Tito.',
 'De Biljana Pekusic, din Belgrad pentru Southeast European Times -- 07/09/11',
 'Miniştrii de externe şi şefii delegaţiilor Mişcării Ţărilor Nealiniate pozează în faţa clădirii parlamentului sârb din Belgrad în 5 septembrie. [Reuters]']

In [ ]:
from datasets import load_metric


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

metric = load_metric("sacrebleu")

predictions, references = postprocess_text(predicted_en_ro, test_dataset["ro-sr"]["ro"])
bleu_score = metric.compute(predictions=predictions, references=references)
print(f"BLEU: {bleu_score['score']:.4f}")

results["sr_en_ro"] = bleu_score

BLEU: 25.8353


# Parse sample inputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd 'drive/MyDrive/Colab Notebooks/'

import ast

filename = "results_sample.txt"

f = open(filename, "r")
data = f.read()

b = ast.literal_eval('(' + data + ')')
b[0]["bg"]
# b = [ast.literal_eval(elem) for elem in b]